In [236]:
import pandas as pd
import glob
import os
import numpy as np


In [237]:
# determine which account is which based on the ad name

def map_account(ad_name):

    if "AEF" in ad_name:
        return "AEF"
    elif "FLA" in ad_name:
        return "FLA"
    elif "FLD" in ad_name:
        return "FLD"
    else:
        return "unknown"

def map_creative_type(ad_name):

    if "Image" in ad_name:
        return "Image"
    elif "Video" in ad_name:
        return "Video"

    else:
        return "unknown"

def create_hook(ad_name):

    try:

        if "Image" in ad_name:

            try:
                return ad_name.split('V')[1].split(' - ')[0]

            except:
                return ad_name.split('V')[1]

        
        if "Video" in ad_name:

            try:    
                return ad_name.split('V')[2].split(' - ')[0]

            except:
                return ad_name.split('V')[2]

    except:
        return "unknown or no hook"

def creative_name(ad_name):

    try:

        if "Image" in ad_name:
            return "Image " + ad_name.split('Image')[1].split(' ')[1]

        if "Video" in ad_name:
            return "Video " + ad_name.split('Video')[1].split(' ')[1]

    except:
        return "unknown"

In [238]:
# combine all csv files in the folder

folder_path = "./raw-data"

csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

df = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)

In [239]:
# remove columns that are not needed

columns_to_drop = ['Result type']

df = df.drop(columns_to_drop, axis=1)

In [240]:
# filter the data

spend_filter = df['Amount spent (USD)'] > 0

In [241]:
# Apply filters

df = df[spend_filter]


In [242]:
# add account name column

for col in df.columns:
    df['Account Name'] = df['Ad name'].apply(map_account)

In [243]:
# add creative type column

for col in df.columns:
    df['Creative Type'] = df['Ad name'].apply(map_creative_type)

In [244]:
# add hook column

for col in df.columns:
    df['Hook'] = df['Ad name'].apply(create_hook)


In [245]:
## ad creative name

for col in df.columns:
    df['Creative Name'] = df['Ad name'].apply(creative_name)

In [246]:
df

,Day,Ad name,Amount spent (USD),Results,Impressions,Link clicks,3-second video plays,Purchases conversion value,Video average play time,Video plays at 75%,Video plays,Preview link,Reporting starts,Reporting ends,Account Name,Creative Type,Hook,Creative Name
0,2025-08-13,Ad - AEF FB Image 83 - V18,2486.31,105.0,214273,1353.0,NaN,2895.65,NaN,NaN,NaN,https://www.facebook.com/?feed_demo_ad=1202353...,2025-08-13,2025-08-13,AEF,Image,18,Image 83
1,2025-08-12,Ad - AEF FB Image 72 - V21,2404.78,107.0,94979,1409.0,NaN,2133.77,NaN,NaN,NaN,https://www.facebook.com/?feed_demo_ad=1202325...,2025-08-12,2025-08-12,AEF,Image,21,Image 72
2,2025-08-12,Ad - AEF FB Image 83 - V18,2050.93,119.0,193517,1507.0,NaN,3218.75,NaN,NaN,NaN,https://www.facebook.com/?feed_demo_ad=1202353...,2025-08-12,2025-08-12,AEF,Image,18,Image 83
3,2025-08-12,Ad - AEF FB Image 80 - V25,1954.19,91.0,89180,947.0,NaN,1907.48,NaN,NaN,NaN,https://www.facebook.com/?feed_demo_ad=1202353...,2025-08-12,2025-08-12,AEF,Image,25,Image 80
4,2025-08-12,Ad - AEF FB Image 78 - V23,1793.11,102.0,93169,1021.0,NaN,2570.33,NaN,NaN,NaN,https://www.facebook.com/?feed_demo_ad=1202339...,2025-08-12,2025-08-12,AEF,Image,23,Image 78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,2025-08-12,Ad - FLD FB Video 53 - S1 - V2,0.01,NaN,1,NaN,NaN,NaN,1.0,NaN,1.0,https://www.facebook.com/?feed_demo_ad=1202158...,2025-08-12,2025-08-12,FLD,Video,2,Video 53
2107,2025-08-12,Ad - FLD FB Video 57 - V4,0.03,NaN,5,NaN,NaN,NaN,2.0,NaN,3.0,https://www.facebook.com/?feed_demo_ad=1202164...,2025-08-12,2025-08-12,FLD,Video,4,Video 57
2108,2025-08-12,Ad - FLD FB Video 54 - S1 - V2,0.13,NaN,5,NaN,2.0,NaN,3.0,NaN,4.0,https://www.facebook.com/?feed_demo_ad=1202160...,2025-08-12,2025-08-12,FLD,Video,2,Video 54
2109,2025-08-12,Ad - FLD FB Video 47 - S1 - V1 - T4 - P2 - G1,6.52,NaN,184,4.0,41.0,NaN,3.0,1.0,152.0,https://www.facebook.com/?feed_demo_ad=1202132...,2025-08-12,2025-08-12,FLD,Video,1,Video 47


In [247]:
def create_metrics(df):
    df['ROAS'] = (df['revenue'] / df['amount_spent']).round(2)
    df['CTR'] = ((df['link_clicks'] / df['impressions']) * 100).round(2)
    df['Conversion Rate'] = ((df['results'] / df['link_clicks']) * 100).round(2)
    df['Thumbstop Rate'] = ((df['three_second_video_plays'] / df['impressions']) * 100).round(2)
    return df

In [248]:
def GroupByAccount(df, account, creative_type, group_by):
    # filter first
    df = df[
        (df['Account Name'] == account)
        & (df['Creative Type'] == creative_type)
    ]

    # aggregate just what you need
    grouped = df.groupby(group_by).agg(
        account_name=('Account Name', 'first'),
        amount_spent=('Amount spent (USD)', 'sum'),
        impressions=('Impressions', 'sum'),
        link_clicks=('Link clicks', 'sum'),
        results=('Results', 'sum'),
        revenue=('Purchases conversion value', 'sum'),
        three_second_video_plays=('3-second video plays', 'sum'),
    )

    # calculate metrics
    grouped = create_metrics(grouped)

    grouped = grouped.sort_values(by='amount_spent', ascending=False)
    # then format
    grouped['amount_spent'] = grouped['amount_spent'].apply(lambda x: f"${x:,.2f}")
    grouped['revenue'] = grouped['revenue'].apply(lambda x: f"${x:,.2f}")


    # keep only final columns (instead of aggregating then dropping)
    grouped = grouped[['account_name', 'amount_spent', 'revenue', 'ROAS', 'CTR', 'Conversion Rate', 'Thumbstop Rate']]

    return grouped

In [249]:
GroupByAccount(df, 'AEF', 'Video', 'Hook')

,account_name,amount_spent,revenue,ROAS,CTR,Conversion Rate,Thumbstop Rate
Hook,,,,,,,
1,AEF,"$13,812.56","$13,542.27",0.98,1.28,8.60,20.42
2,AEF,"$11,300.06","$13,007.06",1.15,0.96,8.33,24.71
4,AEF,"$7,758.50","$11,632.29",1.50,0.84,19.23,16.96
3,AEF,"$7,184.70","$13,043.23",1.82,0.99,22.18,12.98
5,AEF,"$5,948.40","$4,791.15",0.81,1.28,6.96,22.37
6,AEF,"$3,647.11","$3,375.09",0.93,1.05,9.04,22.17
00,AEF,"$3,290.30","$3,590.43",1.09,1.11,9.32,22.62
001,AEF,"$1,504.48","$1,249.55",0.83,1.71,6.75,25.74
5.1,AEF,$895.47,"$1,103.13",1.23,0.72,10.12,16.38


In [ ]:
## export csv here to sanity check
#df.to_csv("output.csv", index=False)